In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
df_crimes = pd.read_csv("data/isp/SeriesHistoricas.csv", sep=';')
df_crimes.head()

,ano,area geografica,delito,contagem,populacao,taxa por 100 mil habitantes
0,1991,Estado,Letalidade Violenta,7.723,12.807.706,"60,3"
1,1992,Estado,Letalidade Violenta,7.923,12.975.915,"61,1"
2,1993,Estado,Letalidade Violenta,7.985,13.146.333,"60,7"
3,1994,Estado,Letalidade Violenta,8.631,13.318.990,"64,8"
4,1995,Estado,Letalidade Violenta,8.638,13.493.914,"64,0"


In [82]:
df_crimes = df_crimes.rename(columns={'taxa por 100 mil habitantes':'taxa_por_100_mil_habitantes', 'area geografica': 'area_geografica'})
df_crimes

,ano,area_geografica,delito,contagem,populacao,taxa_por_100_mil_habitantes
0,1991,Estado,Letalidade Violenta,7.723,12.807.706,"60,3"
1,1992,Estado,Letalidade Violenta,7.923,12.975.915,"61,1"
2,1993,Estado,Letalidade Violenta,7.985,13.146.333,"60,7"
3,1994,Estado,Letalidade Violenta,8.631,13.318.990,"64,8"
4,1995,Estado,Letalidade Violenta,8.638,13.493.914,"64,0"
...,...,...,...,...,...,...
378,2018,Interior,Morte por Intervencao do Agente do Estado,190.000,4.851.156,"3,9"
379,2019,Interior,Morte por Intervencao do Agente do Estado,209.000,4.895.523,"4,3"
380,2020,Interior,Morte por Intervencao do Agente do Estado,176.000,4.938.306,"3,6"
381,2021,Interior,Morte por Intervencao do Agente do Estado,179.000,4.979.358,"3,6"


In [83]:
df_pol = pd.read_csv('data/isp/PoliciaisMortos.csv', sep=';')
df_pol.head()

,CISP,mes,vano,pol_militares_mortos_serv,pol_civis_mortos_serv
0,1,1,2003,0,0
1,4,1,2003,0,0
2,5,1,2003,0,0
3,6,1,2003,0,0
4,7,1,2003,0,0


In [84]:
df_pol = df_pol.rename(columns={'vano': 'ano'})

#### Taxa por 100 mil habitantes
É a métrica mais importante provavelmente pra analise de BI. É a medida prazoniada que responde da pergunta: quantos delites ocorrem pra cada grupo de 100 mil pessoas;

CIdade A 100 crimes/50k pessoas

#### Populacao
É o número total de habitantes estimado pra aquela area geografica (o "Estado") naquele ano específico.

#### Contagem 
É o número bruto e absoluto de ocrrencias registradas pra um determinado delito em um determinado local.
Ex: contagem = 7.723, significa que houve 7.723 eventos (mortes) classificadas como letalidade violenta no estao inteiro durante o ano
de 1991.

NO BI posso lidar com isso como um FATO (ou metrica).


Transformando as colunas contagem_pm_mortos e contagem_pc_mortos em linhas

In [85]:
df_policiais_agg = df_pol.groupby('ano').agg(
    Obitos_PM_Servico=('pol_militares_mortos_serv', 'sum'),
    Obitos_PC_Servico=('pol_civis_mortos_serv', 'sum')
).reset_index()

df_policiais_agg

,ano,Obitos_PM_Servico,Obitos_PC_Servico
0,2003,43,7
1,2004,50,0
2,2005,24,9
3,2006,27,2
4,2007,23,9
5,2008,22,4
6,2009,24,7
7,2010,15,5
8,2011,7,5
9,2012,14,4


In [86]:
df_final_large = pd.merge(
    df_crimes,
    df_policiais_agg,
    on='ano',
    how='left'
)

df_final_large['Obitos_PM_Servico'] = df_final_large['Obitos_PM_Servico'].fillna(0)
df_final_large['Obitos_PC_Servico'] = df_final_large['Obitos_PC_Servico'].fillna(0)

df_final_large = df_final_large.astype({
    'Obitos_PM_Servico': int,
    'Obitos_PC_Servico': int
})

df_final_large

,ano,area_geografica,delito,contagem,populacao,taxa_por_100_mil_habitantes,Obitos_PM_Servico,Obitos_PC_Servico
0,1991,Estado,Letalidade Violenta,7.723,12.807.706,"60,3",0,0
1,1992,Estado,Letalidade Violenta,7.923,12.975.915,"61,1",0,0
2,1993,Estado,Letalidade Violenta,7.985,13.146.333,"60,7",0,0
3,1994,Estado,Letalidade Violenta,8.631,13.318.990,"64,8",0,0
4,1995,Estado,Letalidade Violenta,8.638,13.493.914,"64,0",0,0
...,...,...,...,...,...,...,...,...
378,2018,Interior,Morte por Intervencao do Agente do Estado,190.000,4.851.156,"3,9",28,4
379,2019,Interior,Morte por Intervencao do Agente do Estado,209.000,4.895.523,"4,3",22,0
380,2020,Interior,Morte por Intervencao do Agente do Estado,176.000,4.938.306,"3,6",14,5
381,2021,Interior,Morte por Intervencao do Agente do Estado,179.000,4.979.358,"3,6",15,2


In [87]:
df_final_large['delito'] = df_final_large['delito'].replace(
    'Homic�dio Doloso', 'Homicidio Doloso'
)

df_final_large.sort_values(by=['ano', 'delito'], inplace=True)
df_final_large.reset_index(inplace=True, drop=True)

In [88]:
df_final_large.to_csv('data_processed/pol_pop_death_per_year.csv', sep=';', decimal=',', index=False)
df_final_large.tail(10)

,ano,area_geografica,delito,contagem,populacao,taxa_por_100_mil_habitantes,Obitos_PM_Servico,Obitos_PC_Servico
373,2022,Estado,Letalidade Violenta,4.485,17.561.373,"25,5",17,2
374,2022,Capital,Letalidade Violenta,1.319,6.803.421,"19,4",17,2
375,2022,Baixada Fluminense,Letalidade Violenta,1.415,3.942.439,"35,9",17,2
376,2022,Grande Niteroi,Letalidade Violenta,444.000,1.794.560,"24,7",17,2
377,2022,Interior,Letalidade Violenta,1.307,5.020.953,"26,0",17,2
378,2022,Estado,Morte por Intervencao do Agente do Estado,1.330,17.561.373,"7,6",17,2
379,2022,Capital,Morte por Intervencao do Agente do Estado,444.000,6.803.421,"6,5",17,2
380,2022,Baixada Fluminense,Morte por Intervencao do Agente do Estado,476.000,3.942.439,"12,1",17,2
381,2022,Grande Niteroi,Morte por Intervencao do Agente do Estado,210.000,1.794.560,"11,7",17,2
382,2022,Interior,Morte por Intervencao do Agente do Estado,200.000,5.020.953,"4,0",17,2
